In [3]:
import sqlite3
import pandas as pd
import re 
from datetime import datetime
import os  # Import the os module

# Specify the full path to the database file for the Kindle Vocabulary Builder 
db_file_path = "/Volumes/Kindle/system/vocabulary/vocab.db"

connection = sqlite3.connect(db_file_path)
cursor = connection.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)

for table in tables:
    cursor.execute(f"SELECT * FROM {table[0]} LIMIT 1;")
    columns = cursor.description
    print(f"Table: {table[0]}")
    for column in columns:
        print(column[0])
    print('\n')

connection.close()

df_words = pd.DataFrame()
df_usage = pd.DataFrame()


import sqlite3
import pandas as pd

with sqlite3.connect('/Volumes/Kindle/system/vocabulary/vocab.db') as connection:
    cursor = connection.cursor()
    df_words = pd.read_sql_query("SELECT word, stem FROM WORDS;", connection)
    df_usage = pd.read_sql_query("SELECT word_key, usage, dict_key, book_key FROM LOOKUPS;", connection)
    df_books = pd.read_sql_query("SELECT id, lang, authors, title FROM BOOK_INFO;", connection)
    # Empty the 'words' and 'lookups' tables
    # This will delete all of the data in the tables permanently!
    # cursor.execute("TRUNCATE TABLE words;")
    # cursor.execute("TRUNCATE TABLE lookups;")
    
    
    # Commit the changes
    connection.commit()
# Extract and assign the 'word_key' values to the new DataFrame
df_usage['word_key'] = df_usage['word_key'].str.extract(r":(\w+)")

result = pd.merge(df_usage, df_books, left_on='book_key', right_on='id', how='inner')
result = pd.merge(result, df_words, left_on='word_key', right_on='stem', how='inner')

[('WORDS',), ('LOOKUPS',), ('BOOK_INFO',), ('DICT_INFO',), ('METADATA',), ('VERSION',)]
Table: WORDS
id
word
stem
lang
category
timestamp
profileid


Table: LOOKUPS
id
word_key
book_key
dict_key
pos
usage
timestamp


Table: BOOK_INFO
id
asin
guid
lang
title
authors


Table: DICT_INFO
id
asin
langin
langout


Table: METADATA
id
dsname
sscnt
profileid


Table: VERSION
id
dsname
value




DatabaseError: database disk image is malformed

In [ ]:
from langdetect import detect

# Define a function to detect the language of a string
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

# Apply the detect_language function to the usage column of the result DataFrame
result['lang'] = result['usage'].apply(detect_language)

In [ ]:
result = result.rename(columns={'word_key': 'word'})

result=result[['stem', 'word', 'usage', 'title', 'authors', 'lang']].copy()
result.to_csv('/Users/moritzvitt/Desktop/result.csv', index=False)